In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import nltk
# Launch the installer to download "movie_reviews" corpora.
#nltk.download()

For this assignment, I will model the nltk movie reviews data using bag of words and tf idf.

In [3]:
from nltk.corpus import movie_reviews, stopwords

# Grab and process the raw data

positive_reviews = []
negative_reviews = []
    
for fileid in movie_reviews.fileids('pos'):
        positive_reviews.append((movie_reviews.raw(fileid)))
        
for fileid in movie_reviews.fileids('neg'):
        negative_reviews.append((movie_reviews.raw(fileid)))

In [4]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub('\\\\', '', text)
    text = re.sub('\d', '', text)
    text = ' '.join(text.split())
    return text
    
positive = text_cleaner(str(positive_reviews)[:int(len(str(positive_reviews))/100)])
negative = text_cleaner(str(negative_reviews)[:int(len(str(negative_reviews))/100)])

In [5]:
import en_core_web_sm
nlp = en_core_web_sm.load()
positive_doc = nlp(positive)
negative_doc = nlp(negative)

In [6]:
# Group into sentences.
positive_sents = [[sent, "positive"] for sent in positive_doc.sents]
negative_sents = [[sent, "negative"] for sent in negative_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(positive_sents + negative_sents)
sentences.head()

,0,1
0,"([, ', films, adapted, from, comic, books, hav...",positive
1,"(nfor, starters, ,, it, was, created, by, alan...",positive
2,"(nto, say, moore, and, campbell, thoroughly, r...",positive
3,"(the, ripper, would, be, like, saying, michael...",positive
4,"(nthe, book, (, or, "", graphic, novel, ,, "", i...",positive


In [7]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['sentiment'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
positivewords = bag_of_words(positive_doc)
negativewords = bag_of_words(negative_doc)

# Combine bags to create a set of unique words.
common_words = set(positivewords + negativewords)

In [8]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
Processing row 600
Processing row 650
Processing row 700
Processing row 750


,bored,rabie,ndid,nevent,ambulance,care,equally,nis,prim,fischer,...,witted,salary,dangerous,episode,young,jock,kick,contrivance,text_sentence,sentiment
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"([, ', films, adapted, from, comic, books, hav...",positive
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(nfor, starters, ,, it, was, created, by, alan...",positive
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(nto, say, moore, and, campbell, thoroughly, r...",positive
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(the, ripper, would, be, like, saying, michael...",positive
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(nthe, book, (, or, "", graphic, novel, ,, "", i...",positive


In [9]:
rfc = ensemble.RandomForestClassifier()
Y = word_counts['sentiment']
X = np.array(word_counts.drop(['text_sentence','sentiment'], 1))
rfc.fit(X, Y)
score = cross_val_score(rfc, X, Y, cv=5, scoring = 'accuracy')
print(score)
print('Average Accuracy: %0.2f (+/- %0.2f)' % (score.mean(), score.std() * 2))

[0.52830189 0.60126582 0.60126582 0.53797468 0.53164557]
Average Accuracy: 0.56 (+/- 0.07)


In [10]:
lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
lr.fit(X, Y)
score = cross_val_score(lr, X, Y, cv=5, scoring = 'accuracy')
print(score)
print('Average Accuracy: %0.2f (+/- %0.2f)' % (score.mean(), score.std() * 2))

[0.55974843 0.58860759 0.61392405 0.58227848 0.70253165]
Average Accuracy: 0.61 (+/- 0.10)


In [11]:
clf = ensemble.GradientBoostingClassifier()
clf.fit(X, Y)
score = cross_val_score(clf, X, Y, cv=5, scoring = 'accuracy')
print(score)
print('Average Accuracy: %0.2f (+/- %0.2f)' % (score.mean(), score.std() * 2))

[0.49685535 0.55696203 0.67088608 0.60126582 0.66455696]
Average Accuracy: 0.60 (+/- 0.13)


In [12]:
svc = SVC()
svc.fit(X, Y)
score = cross_val_score(svc, X, Y, cv=5, scoring = 'accuracy')
print(score)
print('Average Accuracy: %0.2f (+/- %0.2f)' % (score.mean(), score.std() * 2))

[0.55974843 0.50632911 0.59493671 0.48101266 0.69620253]
Average Accuracy: 0.57 (+/- 0.15)


In [38]:
from sklearn.datasets import load_files
moviedir = r'C:\\Users\\rkies\\AppData\\Roaming\\nltk_data\\corpora\\movie_reviews'

# loading all files. 
movie = load_files(moviedir, shuffle=True)

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
docs_train, docs_test, y_train, y_test = train_test_split(movie.data, movie.target, 
                                                          test_size = 0.20, random_state = 12)
movieVzer= CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize, max_features=3000) # use top 3000 words only. 78.25% acc.
# movieVzer = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize)         # use all 25K words. Higher accuracy

# fit and tranform using training text 
docs_train_counts = movieVzer.fit_transform(docs_train)

In [43]:
# Convert raw frequency counts into TF-IDF values
from sklearn.feature_extraction.text import TfidfTransformer
movieTfmer = TfidfTransformer()
docs_train_tfidf = movieTfmer.fit_transform(docs_train_counts)

In [44]:
# Using the fitted vectorizer and transformer, tranform the test data
docs_test_counts = movieVzer.transform(docs_test)
docs_test_tfidf = movieTfmer.transform(docs_test_counts)

In [45]:
rfc = ensemble.RandomForestClassifier()
rfc.fit(docs_train_tfidf, y_train)
score = cross_val_score(rfc, docs_train_tfidf, y_train, cv=5, scoring = 'accuracy')
print(score)
print('Average Accuracy: %0.2f (+/- %0.2f)' % (score.mean(), score.std() * 2))

[0.79375  0.825    0.79375  0.821875 0.709375]
Average Accuracy: 0.79 (+/- 0.08)


In [46]:
lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
lr.fit(docs_train_tfidf, y_train)
score = cross_val_score(lr, docs_train_tfidf, y_train, cv=5, scoring = 'accuracy')
print(score)
print('Average Accuracy: %0.2f (+/- %0.2f)' % (score.mean(), score.std() * 2))

[0.815625 0.825    0.759375 0.7875   0.76875 ]
Average Accuracy: 0.79 (+/- 0.05)


In [47]:
clf = ensemble.GradientBoostingClassifier()
clf.fit(docs_train_tfidf, y_train)
score = cross_val_score(clf, docs_train_tfidf, y_train, cv=5, scoring = 'accuracy')
print(score)
print('Average Accuracy: %0.2f (+/- %0.2f)' % (score.mean(), score.std() * 2))

[0.8      0.809375 0.778125 0.825    0.7625  ]
Average Accuracy: 0.80 (+/- 0.04)


In [49]:
clf = ensemble.GradientBoostingClassifier(n_estimators = 250)
clf.fit(docs_train_tfidf, y_train)
score = cross_val_score(clf, docs_train_tfidf, y_train, cv=5, scoring = 'accuracy')
print(score)
print('Average Accuracy: %0.2f (+/- %0.2f)' % (score.mean(), score.std() * 2))

[0.8      0.81875  0.80625  0.84375  0.771875]
Average Accuracy: 0.81 (+/- 0.05)
